# ✍️ QualiVault: Transcribe
**Goal:** Transcribe processed FLAC files using Whisper and Pyannote.

1. Loads `processing_recipe.yaml`.
2. Iterates through 'pending' interviews.
3. Runs Diarization (Speaker ID) and Transcription.
4. Uses 'Hybrid' logic (Left/Right channel check) if separation is good.
5. Saves a CSV transcript for each interview.

In [ ]:
%load_ext autoreload
%autoreload 2
import yaml
from pathlib import Path
from qualivault.core import setup_environment
from qualivault import find_workspace_root

# 1. Setup Environment (Load Token & Config)
setup_environment()

# 2. Load Config Paths
PROJECT_NAME = 'YOUR_PROJECT_NAME'  # <-- Change this to work on different projects
workspace_root = find_workspace_root(Path.cwd())
project_root = workspace_root / 'projects' / PROJECT_NAME
config_path = project_root / 'config.yml'

if not project_root.exists():
    raise FileNotFoundError(f"❌ Project not found: {project_root}")
if not config_path.exists():
    raise FileNotFoundError(f"❌ Config not found: {config_path}")

with open(config_path) as f:
    config = yaml.safe_load(f)

recipe_path = project_root / "processing_recipe.yaml"

print(f"📁 Project: {project_root.name}")
print(f"📄 Recipe: {recipe_path}")
print(f"⚙️  Transcription backend: {config.get('transcription', {}).get('backend', 'auto')}")
print(f"🤖 Model: {config.get('transcription', {}).get('model_id', 'openai/whisper-large-v3')}")

In [ ]:
# 3. Run Transcription Loop (resumable)
from qualivault.pipeline import transcribe_recipe

RESUME = True          # True: skips successful transcripts, continues after failures
MAX_INTERVIEWS = None  # e.g. 2 for testing, None for all

transcribe_recipe(
    project_root=project_root,
    config=config,
    recipe_path=recipe_path,
    resume=RESUME,
    max_items=MAX_INTERVIEWS,
)